To execute the commands and view the output as shown in this webpage, execute the following command from command prompt

ipython notebook

It should open up a webpage and from there create a new notebook.

Note: Page at http://ipython.org/notebook.html provides good introductory information about ipython and its usage.

In [1]:
import sys
#sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')
sys.path.append('/scratch/pradap/python-work/enrique/')

In [2]:
import magellan as mg

DEBUG:Cloud:Log file (/u/p/r/pradap/.picloud/cloud.log) opened


In [3]:
mg.init_jvm()
#mg.init_jvm('/Library/Java/JavaVirtualMachines/jdk1.8.0_45.jdk/Contents/Home/jre/lib/server/libjvm.dylib')

True

In [4]:
# get filepaths for tables A and B. 
path_A = mg.get_install_path() + '/datasets/table_A.csv' # path_A is the file path where table_A.csv is stored
path_B = mg.get_install_path() + '/datasets/table_B.csv' # path_B is the file path where table_B.csv is stored

In [5]:
# read table A; table A has 'ID as the key attribute
A = mg.read_csv(path_A, key='ID')
B = mg.read_csv(path_B, key='ID')

In [6]:
A

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


In [7]:
B

,ID,name,birth_year,hourly_wage,address,zipcode
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122
5,b6,Michael Brodie,1987,32.5,"133 Clement Street, San Francisco",94107


In [8]:
# view table A
#mg.view(A)

In [9]:
# Blocking
# plan : 
# A, B --overlap blocking---> candset --attr-equiv-block--> candset|
# A, B ------------rule-based-blocking--------------------> candset|----union--->candset
# A, B -----------black-box-blocking----------------------> candset|


In [10]:
# Overlap blocking over input tables
ob = mg.OverlapBlocker()
# block using zipcode
C = ob.block_tables(A, B, 'name', 'name', l_output_attrs=['zipcode', 'name', 'birth_year'], 
                    r_output_attrs=['zipcode', 'name', 'birth_year'], overlap_size=1)
C

0%  100%
[######] | ETA[sec]: 0.000 
Total time elapsed: 0.018 sec


,_id,ltable.ID,rtable.ID,ltable.zipcode,ltable.name,ltable.birth_year,rtable.zipcode,rtable.name,rtable.birth_year
0,0,a2,b3,94122,Michael Franklin,1988,94122,Mike Franklin,1988
1,1,a2,b6,94122,Michael Franklin,1988,94107,Michael Brodie,1987
2,2,a3,b2,94107,William Bridge,1986,94107,Bill Bridge,1986
3,3,a5,b5,94122,Alphonse Kemper,1984,94122,Alfons Kemper,1984


In [11]:
# Attribute equivalence blocking: block C using birth_year
ab = mg.AttrEquivalenceBlocker()
D = ab.block_candset(C, 'birth_year', 'birth_year')

0%  100%
[####] | ETA[sec]: 0.000 
Total time elapsed: 0.004 sec


In [12]:
D

,_id,ltable.ID,rtable.ID,ltable.zipcode,ltable.name,ltable.birth_year,rtable.zipcode,rtable.name,rtable.birth_year
0,0,a2,b3,94122,Michael Franklin,1988,94122,Mike Franklin,1988
2,2,a3,b2,94107,William Bridge,1986,94107,Bill Bridge,1986
3,3,a5,b5,94122,Alphonse Kemper,1984,94122,Alfons Kemper,1984


In [13]:
# Rule-based blocking over input tables
# first get features that can be used
feature_table = mg.get_features_for_blocking(A, B)

In [39]:
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,ID_ID_lev,ID,ID,None,None,lev,<function ID_ID_lev at 0x7f1ef44dacf8>,from magellan.feature.simfunctions import *\nf...
1,ID_ID_jar,ID,ID,None,None,jaro,<function ID_ID_jar at 0x7f1ee69c77d0>,from magellan.feature.simfunctions import *\nf...
2,ID_ID_jwn,ID,ID,None,None,jaro_winkler,<function ID_ID_jwn at 0x7f1ee69c7758>,from magellan.feature.simfunctions import *\nf...
3,ID_ID_sdx,ID,ID,None,None,soundex,<function ID_ID_sdx at 0x7f1ee69c7938>,from magellan.feature.simfunctions import *\nf...
4,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x7f1ee69c78c0>,from magellan.feature.simfunctions import *\nf...
5,ID_ID_jac_qgm_3_qgm_3,ID,ID,qgm_3,qgm_3,jaccard,<function ID_ID_jac_qgm_3_qgm_3 at 0x7f1ee69c7...,from magellan.feature.simfunctions import *\nf...
6,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x7f1ee...,from magellan.feature.simfunctions import *\nf...
7,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x7...,from magellan.feature.simfunctions import *\nf...
8,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x7...,from magellan.feature.simfunctions import *\nf...
9,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x7f1ee69c7500>,from magellan.feature.simfunctions import *\nf...


In [42]:
# Create rule-based blocker
rb = mg.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.9
rb.add_rule(['name_name_lev(ltuple, rtuple) < 0.9'], feature_table)

True

In [43]:
E = rb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'])

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.022 sec


In [44]:
E

,_id,ltable.ID,rtable.ID,ltable.name,rtable.name


In [45]:
dbg = mg.debug_blocker(A, B, E)

0%  100%
[######] | ETA[sec]: 0.000 
Total time elapsed: 0.003 sec


In [46]:
dbg.head(5)

,_id,similarity,ltable.ID,rtable.ID,ltable.name,rtable.name,ltable.address,rtable.address
0,0,0.804348,a3,b2,William Bridge,Bill Bridge,"3131 Webster St, San Francisco","3131 Webster St, San Francisco"
1,1,0.760870,a2,b3,Michael Franklin,Mike Franklin,"1652 Stockton St, San Francisco","1652 Stockton St, San Francisco"
2,2,0.467742,a5,b5,Alphonse Kemper,Alfons Kemper,"1702 Post Street, San Francisco","170 Post St, Apt 4, San Francisco"
3,3,0.321429,a1,b3,Kevin Smith,Mike Franklin,"607 From St, San Francisco","1652 Stockton St, San Francisco"
4,4,0.313433,a2,b6,Michael Franklin,Michael Brodie,"1652 Stockton St, San Francisco","133 Clement Street, San Francisco"


In [47]:
# Create rule-based blocker --- NOTE: we are creating a new blocker !!!
rb = mg.RuleBasedBlocker()
# Add rule : block tuples if name_name_lev(ltuple, rtuple) < 0.4
rb.add_rule(['name_name_lev(ltuple, rtuple) < 0.4'], feature_table)

True

In [48]:
E = rb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'])

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.032 sec


In [49]:
E

,_id,ltable.ID,rtable.ID,ltable.name,rtable.name
0,0,a2,b3,Michael Franklin,Mike Franklin
1,1,a2,b6,Michael Franklin,Michael Brodie
2,2,a3,b2,William Bridge,Bill Bridge
3,3,a3,b6,William Bridge,Michael Brodie
4,4,a4,b2,Binto George,Bill Bridge
5,5,a5,b5,Alphonse Kemper,Alfons Kemper


In [23]:
# Apply black box blocker
# Create black box blocker
bb = mg.BlackBoxBlocker()

In [24]:
# define a black box function. 
# The blocker function would like to drop tuples whose last name do not match
# The function has to do the following steps
#  1) Get name attributes from each of the tuples
#  2) Split name attribute to get last name
#  3) if last names donot match return True

In [25]:
def my_function(x, y):
    # x, y will be of type pandas series
    
    # get name attribute
    x_name = x['name']
    y_name = y['name']
    # get last names
    x_name = x_name.split(' ')[1]
    y_name = y_name.split(' ')[1]
    # check if last names match
    if x_name != y_name:
        return True
    else:
        return False

In [26]:
bb.set_black_box_function(my_function)

True

In [27]:
F = bb.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'])

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.023 sec


In [28]:
F

,_id,ltable.ID,rtable.ID,ltable.name,rtable.name
0,0,a2,b3,Michael Franklin,Mike Franklin
1,1,a3,b2,William Bridge,Bill Bridge
2,2,a5,b5,Alphonse Kemper,Alfons Kemper


In [29]:
# Combine all the blocker outputs

G = mg.combine_block_outputs_via_union([D, E, F])

In [30]:
G

,_id,ltable.ID,rtable.ID,ltable.birth_year,ltable.name,ltable.zipcode,rtable.birth_year,rtable.name,rtable.zipcode
0,0,a2,b3,1988,Michael Franklin,94122,1988,Mike Franklin,94122
1,1,a2,b6,1988,Michael Franklin,94122,1987,Michael Brodie,94107
2,2,a3,b2,1986,William Bridge,94107,1986,Bill Bridge,94107
3,3,a3,b6,1986,William Bridge,94107,1987,Michael Brodie,94107
4,4,a5,b5,1984,Alphonse Kemper,94122,1984,Alfons Kemper,94122


In [31]:
# export G
G.to_csv('combined_result.csv')

True

In [32]:
# read back G
G1 = mg.read_csv('combined_result.csv', ltable=A, rtable=B)

In [33]:
G1

,_id,ltable.ID,rtable.ID,ltable.birth_year,ltable.name,ltable.zipcode,rtable.birth_year,rtable.name,rtable.zipcode
0,0,a2,b3,1988,Michael Franklin,94122,1988,Mike Franklin,94122
1,1,a2,b6,1988,Michael Franklin,94122,1987,Michael Brodie,94107
2,2,a3,b2,1986,William Bridge,94107,1986,Bill Bridge,94107
3,3,a3,b6,1986,William Bridge,94107,1987,Michael Brodie,94107
4,4,a5,b5,1984,Alphonse Kemper,94122,1984,Alfons Kemper,94122


In [34]:
# save rule-based blocker

In [35]:
mg.save_object(rb,'rule-based-blocker.pkl')

True

In [36]:
rb1 = mg.load_object('rule-based-blocker.pkl')

In [37]:
E1 = rb1.block_tables(A, B, l_output_attrs=['name'], r_output_attrs=['name'])

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.030 sec


In [38]:
E1

,_id,ltable.ID,rtable.ID,ltable.name,rtable.name
0,0,a2,b3,Michael Franklin,Mike Franklin
1,1,a2,b6,Michael Franklin,Michael Brodie
2,2,a3,b2,William Bridge,Bill Bridge
3,3,a3,b6,William Bridge,Michael Brodie
4,4,a5,b5,Alphonse Kemper,Alfons Kemper
